In [ ]:
# AIRE Micro Tutor Identity Cell
import datetime as dt
learner_id = input('Enter your learner ID (email or handle): ').strip()
learner_role = input('Enter your learner role (e.g., researcher, educator): ').strip()
resource_id = 'notebook:tabular_modeling'
session_id = f"{learner_id}-{dt.datetime.utcnow().isoformat()}"

In [ ]:
# AIRE Micro Tutor Imports
from prompt_tutor import evaluate_prompt
from aire_telemetry import log_event
from resources_map import RESOURCE_MAP

In [ ]:
import sys, subprocess, os
from pathlib import Path

# Colab Setup
if "google.colab" in sys.modules:
    print("Running in Google Colab. Installing dependencies...")
    subprocess.run(["pip", "install", "-q", "pandas", "numpy", "scikit-learn", "requests", "pydantic", "jsonschema", "plotly", "tqdm"])
    
    # Check for data
    if not (Path.cwd() / "data").exists():
        print("Data directory not found. Cloning repository...")
        subprocess.run(["git", "clone", "https://github.com/aire-program/aire-researcher-sandbox.git", "_repo"])
        
        # Move data and scripts to current directory
        if (Path("_repo/data").exists()):
            print("Moving data and scripts...")
            subprocess.run(["mv", "_repo/data", "."])
            subprocess.run(["mv", "_repo/scripts", "."])
            subprocess.run(["rm", "-rf", "_repo"])
        else:
            print("Warning: Data not found in cloned repo.")
    else:
        print("Data directory found.")


# Engineer Features from Experiments

**What**: Derive new features from existing experimental data to enhance analytical power.

**Why**: Raw data often lacks the specific signals needed for modeling. Feature engineering transforms raw variables into more meaningful representations.

**How**:
1. **Normalize metrics** (Z-score standardization).
2. **Extract temporal features** (e.g., day of week).
3. **Create binary indicators** for categorical variables.

**Key Concept**: **Feature Engineering** is the process of using domain knowledge to extract features from raw data that make machine learning algorithms work better.

By the end of this notebook, you will have completed the listed steps and produced the outputs described in the success criteria.

### Success criteria
- You computed normalized metrics.
- You derived categorical/time features.
- You prepared a feature-rich table for downstream use.

In [ ]:
from pathlib import Path


def find_data_dir() -> Path:
    candidates = [Path.cwd() / "data", Path.cwd().parent / "data", Path.cwd().parent.parent / "data"]
    for candidate in candidates:
        if (candidate / "sample_texts" / "articles_sample.csv").exists():
            return candidate
    raise FileNotFoundError("data directory not found. Run scripts/generate_synthetic_data.py.")

DATA_DIR = find_data_dir()


In [ ]:
import pandas as pd

experiments = pd.read_csv(DATA_DIR / "sample_tabular" / "experiments_sample.csv")
experiments["timestamp"] = pd.to_datetime(experiments["timestamp"])

experiments["metric_normalized"] = (experiments["metric_value"] - experiments["metric_value"].mean()) / experiments["metric_value"].std()
experiments["day_of_week"] = experiments["timestamp"].dt.day_name()
experiments["is_treatment"] = experiments["condition"].str.contains("treatment")
experiments[["experiment_id", "condition", "metric_value", "metric_normalized", "day_of_week", "is_treatment"]].head()


### If you get stuck / What to try next

If you get stuck: verify the cleaning notebook ran and that dependencies installed. What to try next: prototype simple models or export features to the notebook outputs for quick views.

In [ ]:
# AIRE Micro Tutor Prompt + Feedback Cell
from aire_llm_client import chat_completion

def call_llm(prompt_text: str) -> str:
    messages = [
        {'role': 'system', 'content': 'You are an AI helper responding concisely.'},
        {'role': 'user', 'content': prompt_text},
    ]
    return chat_completion(messages)

prompt_text = input('Enter the prompt you want to test: ').strip()
ai_response = call_llm(prompt_text)
evaluation = evaluate_prompt(prompt_text, learner_role)

clarity = evaluation.get('clarity_score')
context = evaluation.get('context_score')
constraints = evaluation.get('constraints_score')
evaluation_score = evaluation.get('evaluation_score')
primary_weakness = evaluation.get('primary_weakness', '')
recommended_resource_id = RESOURCE_MAP.get(primary_weakness) or RESOURCE_MAP.get('evaluation')

log_event(
    event_name='micro_tutor_evaluation',
    user_id=learner_id or session_id,
    metadata={
        'resource_id': resource_id,
        'learner_role': learner_role,
        'clarity': clarity,
        'context': context,
        'constraints': constraints,
        'evaluation_score': evaluation_score,
        'primary_weakness': primary_weakness,
    },
)

print('\n--- AI Response ---')
print(ai_response)
print('\n--- Feedback ---')
print(evaluation.get('summary', 'No summary provided.'))
print('Primary weakness:', primary_weakness or 'n/a')
print('Strengths:', ', '.join(evaluation.get('strengths', [])) or 'n/a')
print('Suggestions:', ', '.join(evaluation.get('suggestions', [])) or 'n/a')
print('\nRecommended resource:', recommended_resource_id or 'n/a')